In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import dalex as dx

In [14]:
# load data
df = pd.read_csv('data/ACSIncome_NY_2018.csv')

### Prepare Data

In [15]:
# rename target for readability
df = df.rename(columns={'TARGET': 'INCOME'})

# keep only certain features
df = df[["AGEP", "COW", "SCHL", "MAR", "WKHP", "SEX", "INCOME"]]


X = df.iloc[:, :-1]
y = df.iloc[:, -1]

### Fit Models

In [16]:
clf_forest = RandomForestClassifier().fit(X, y)
clf_mlp = MLPClassifier().fit(X, y)
clf_logreg = LogisticRegression().fit(X, y)
clf_dt = DecisionTreeClassifier().fit(X, y)

In [17]:
# create Explainer objects 
exp_forest  = dx.Explainer(clf_forest, X,y, verbose = False)
exp_mlp  = dx.Explainer(clf_mlp, X,y, verbose = False)
exp_logreg  = dx.Explainer(clf_logreg, X,y, verbose = False)
exp_dt = dx.Explainer(clf_dt, X,y, verbose = False)

/Users/clarkao1/miniforge3/envs/torch-gpu/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names

/Users/clarkao1/miniforge3/envs/torch-gpu/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but MLPClassifier was fitted with feature names

/Users/clarkao1/miniforge3/envs/torch-gpu/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names

/Users/clarkao1/miniforge3/envs/torch-gpu/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names



### Check Performance

In [18]:
exp_forest.model_performance().result

,recall,precision,f1,accuracy,auc
RandomForestClassifier,0.856999,0.857421,0.85721,0.881587,0.958326


In [19]:
exp_mlp.model_performance().result

,recall,precision,f1,accuracy,auc
MLPClassifier,0.83088,0.663285,0.737683,0.754924,0.849668


In [20]:
exp_logreg.model_performance().result

,recall,precision,f1,accuracy,auc
LogisticRegression,0.6685,0.714486,0.690728,0.751721,0.828011


In [21]:
exp_dt.model_performance().result

,recall,precision,f1,accuracy,auc
DecisionTreeClassifier,0.891474,0.834374,0.861979,0.881597,0.964815


### Create the priviledged and protected groups

In [22]:
df['SEX'] = np.where(df['SEX'] == 2.0, "Female", "Male")

protected = df['SEX']
privileged = "Male"

# create fairness explanations
fobject_forest = exp_forest.model_fairness(protected, privileged)
fobject_mlp = exp_mlp.model_fairness(protected, privileged)
fobject_logreg = exp_logreg.model_fairness(protected, privileged)
fobject_dt = exp_dt.model_fairness(protected, privileged)

### Check for fairness

In [23]:
fobject_forest.fairness_check(epsilon = 0.8) # default epsilon

fobject_forest.plot(objects=[fobject_mlp, fobject_logreg, fobject_dt], epsilon=0.8)

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
Female  0.965517  1.019473  0.987239  0.653226  0.741053


In [24]:
fobject_forest.plot(type = "radar")
fobject_forest.plot(objects=[fobject_mlp, fobject_logreg, fobject_dt], type = "radar")

### Adding noise to the sensitive attributes

In [132]:
# check initial distribution
df['SEX'].value_counts()

male = df['SEX'].value_counts()['Male']
female = df['SEX'].value_counts()['Female']

tot = male + female
print(f'Proportion female {round(female/tot, 3)}')
print(f'Proportion male {round(male/tot, 3)}')

Proportion female 0.494
Proportion male 0.506


Initially, roughly 50-50 distribution!

In [136]:
# oversample the male population
males = df[df['SEX'] == 'Male']
females = df[df['SEX'] == 'Female']